In [ ]:
# Load packages
# Established packages
import os
import numpy as np
import pandas as pd
import geopandas as gp
import datetime
import math
import xarray as xr # labels in the form of dimensions, coordinates and attributes
import matplotlib.pyplot as plt
import zarr

# Custom package covid19model
from covid19model.models import models
from covid19model.data import model_parameters, mobility #, sciensano, google
from covid19model.visualization.output import population_status, infected, show_map, show_graphs
from covid19model.models.time_dependant_parameter_fncs import lockdown_func, mobility_update_func, policies_wave1_4prev

# Moving average for visualisation
from covid19model.visualization.utils import moving_avg
from covid19model.models.utils import read_pops, read_coordinates_nis, initial_state

# Download function for complete calibration
# covid19model.optimization.run_optimization import full_calibration_wave1, full_calibration_wave2


# OPTIONAL: Load the "autoreload" extension so that package code can change
%load_ext autoreload
# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
# This may be useful because the `covid19model` package is under construction
%autoreload 2

# Spatial stratification
agg = 'arr'

In [5]:
initN, Nc_home, Nc_work, Nc_schools, Nc_transport, Nc_leisure, Nc_others, Nc_total = model_parameters.get_interaction_matrices()

In [7]:
Nc_home

array([[7.60755599e-01, 2.63738152e-01, 1.67624352e-01, 1.00581656e+00,
        3.56382432e-01, 8.64238434e-02, 2.44658542e-02, 4.97860003e-03,
        2.58860267e-04],
       [2.58459232e-01, 7.98102855e-01, 2.36229252e-01, 2.09384315e-01,
        1.32831035e+00, 2.08403680e-01, 2.14381199e-02, 1.02639519e-02,
        2.02943185e-04],
       [1.51145984e-01, 2.17357227e-01, 4.94673915e-01, 1.17539487e-01,
        2.90168285e-01, 5.80887721e-01, 6.82418080e-02, 9.03044332e-03,
        2.03603314e-02],
       [8.52386360e-01, 1.81068443e-01, 1.10469402e-01, 5.08076015e-01,
        1.34826577e-01, 1.03636028e-01, 6.34215703e-02, 1.96874250e-02,
        3.33153916e-02],
       [2.69881133e-01, 1.02644741e+00, 2.43695035e-01, 1.20479743e-01,
        6.20731645e-01, 1.31380732e-01, 2.14330776e-02, 3.01626659e-02,
        3.96869834e-02],
       [7.23902642e-02, 1.78128281e-01, 5.39609121e-01, 1.02432938e-01,
        1.45318874e-01, 7.27902019e-01, 1.48161135e-01, 1.41967137e-02,
        2.4